In [5]:
import numpy as np

# CEV process
class ProcessCEV:

    def __init__(self, mu, sigma, gamma):
        self._mu = mu
        self._sigma = sigma
        self._gamma = gamma

    def Simulate(self, T=1, dt=0.001, S0=1.):
        n = round(T / dt)
        
        mu = self._mu
        sigma = self._sigma
        gamma = self._gamma

        gaussian_increments = np.random.normal(size=n - 1)
        res = np.zeros(n)
        res[0] = S0
        S = S0
        sqrt_dt = dt ** 0.5
        for i in range(n - 1):
            S = S + S * mu * dt + sigma * \
                (S ** gamma) * gaussian_increments[i] * sqrt_dt
            res[i + 1] = S

        return res
    
class EstimatorCEV:

    def __init__(self, dt):
        self._dt = dt
        self._alpha0 = -5
    
    def Estimate(self, trajectory):
        sigma, gamma = self._evaluate_sigma_gamma(trajectory, self._alpha0)
        if sigma == None:
            return None, None, None
        else:
            mu = self._estimate_mu(trajectory)
            return (mu, sigma, gamma)

    def _log_increments(self, trajectory):
        return np.diff(trajectory) / trajectory[:-1]
    
    def _estimate_mu(self, trajectory):
        return np.mean(self._log_increments(trajectory)) / self._dt 

    def _log_increments_alpha(self, trajectory, alpha):
        mod_increments = self._log_increments(trajectory ** (1 + alpha))
        return mod_increments / (1 + alpha)

    def _evaluate_Vt(self, trajectory, alpha):
        lhs = self._log_increments_alpha(trajectory, alpha)
        rhs = self._log_increments(trajectory)
        center = 2 * (lhs - rhs) / (alpha * self._dt)
        return center

    def _evaluate_sigma_gamma(self, trajectory, alpha):
        if np.any(trajectory <= 0):
            return None, None
        
        Vts = self._evaluate_Vt(trajectory, alpha)
        if np.any(Vts <= 0):
            return None, None
        logVts = np.log(Vts)

        Sts = trajectory[:-1]  # removes the last term as in eq. (5)
        if np.any(Sts <= 0):
            return None, None
        logSts = np.log(Sts)

        ones = np.ones(Sts.shape[0])
        A = np.column_stack((ones, logSts))

        res = np.linalg.lstsq(A, logVts, rcond=None)[0]
        return (2 * np.exp(res[0] / 2), 0.5 * (res[1] + 2))


if __name__ == "__main__":

    #from ProcessCEV import ProcessCEV

    def test(true_mu, true_sigma, true_gamma):
        dt = 0.001
        T = 10
        
        sample_mu = []
        sample_sigma = []
        sample_gamma = []

        for i in range(100):
            mu_est, sigma_est, gamma_est = EstimatorCEV(dt=dt).Estimate(ProcessCEV(
                true_mu, true_sigma, true_gamma).Simulate(T, dt=dt))
            
            if mu_est != None:
                sample_mu = [mu_est] + sample_mu
                sample_sigma = [sigma_est] + sample_sigma
                sample_gamma = [gamma_est ] + sample_gamma
    
        print("mu : " + str(true_mu) + " \t| est : " + str(np.mean(sample_mu)) + " \t| std : " + str(np.std(sample_mu)))
        print("sigma : " + str(true_sigma) + " \t| est : " + str(np.mean(sample_sigma)) + " \t| std : " + str(np.std(sample_sigma)))
        print("gamma : " + str(true_gamma) + " \t| est : " + str(np.mean(sample_gamma)) + " \t| std : " + str(np.std(sample_gamma)))
        print(10*"-")


    test(0.,0.5,0.8)
    test(0.2,0.5,0.8)
    test(0.2,0.5,1.2)
    test(0.,0.3,0.2)
    test(0.,0.5,2)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14940\1057563388.py:25: RuntimeWarning: invalid value encountered in scalar power
  (S ** gamma) * gaussian_increments[i] * sqrt_dt


mu : 0.0 	| est : 0.04569706512718758 	| std : 0.13249717363946364
sigma : 0.5 	| est : 0.5312780325979866 	| std : 0.008776204406776858
gamma : 0.8 	| est : 0.802234701244892 	| std : 0.02224648296421392
----------
mu : 0.2 	| est : 0.19247364450120139 	| std : 0.1436549676033203
sigma : 0.5 	| est : 0.5301289349326547 	| std : 0.009895785305198475
gamma : 0.8 	| est : 0.8057035405386296 	| std : 0.02046248460440612
----------
mu : 0.2 	| est : 0.19257270860242795 	| std : 0.19294648507793216
sigma : 0.5 	| est : 0.5285404840377141 	| std : 0.010054761767698905
gamma : 1.2 	| est : 1.1976258988322774 	| std : 0.0233553204822371
----------
mu : 0.0 	| est : 0.07491219877578498 	| std : 0.05813538209410872
sigma : 0.3 	| est : 0.3173187595842497 	| std : 0.006749587140431266
gamma : 0.2 	| est : 0.20740704879088312 	| std : 0.038637557250676406
----------
mu : 0.0 	| est : -0.007175909099417019 	| std : 0.1129058177111101
sigma : 0.5 	| est : 0.5319639709131625 	| std : 0.01401399268864